# **Imports**

In [1]:
import pandas as pd
from openpyxl import load_workbook
from dataclasses import dataclass
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime, timedelta
import openmeteo_requests
import requests_cache
import pandas as pd
from retry_requests import retry
import requests
import bs4
from tqdm import tqdm

# **Constants**

In [2]:
@dataclass(frozen=True)
class Pathes:
    path_turn_on_off_heating: str = '../main_datasets/processed_datasets/6. Плановые-Внеплановые отключения 01.10.2023-30.04.2023.csv'
    path_events_for_period_2023_params: str = '../main_datasets/processed_datasets/События за период_01.10.2023-31.12.2023____pivot_table.csv'
    path_events_for_period_2023: str = '../main_datasets/processed_datasets/События за период_01.10.2023-31.12.2023.xlsx____uploading_table.csv'
    path_events_for_period_2024: str = '../main_datasets/processed_datasets/События_за_период_01.01.2024-30.04.2024____uploading_table.csv'
    path_events_for_period_2024_params: str = '../main_datasets/processed_datasets/События_за_период_01.01.2024-30.04.2024____pivot_table.csv'
    path_to_timeline: str = '../variables/timeline'
    path_to_weather_dataset: str = '../main_datasets/exogens_params/weather_dataset_v2.csv'

In [3]:
date_columns =  [
    'Дата создания во внешней системе', 'Дата закрытия', 'Дата и время завершения события во внешней системе',
    'Дата регистрации отключения', 'Планируемая дата отключения', 'Планируемая дата включения', 'Фактическая дата отключения'
    'Фактическая дата включения'
]

In [4]:
index_month = {
    1: 'января',
    2: 'февраля',
    3: 'марта',
    4: 'апреля',
    10: 'октября',
    11: 'ноября',
    12: 'декабря',
}

In [5]:
lat = 55.787715
lon = 37.775631
max_date_for_api = datetime(year=2024, month=6, day=18)
weather_params = ["temperature_2m", "relative_humidity_2m", "rain", "showers", "snowfall", "cloud_cover", "cloud_cover_low", "cloud_cover_mid", "cloud_cover_high"]

# **Functions**

In [6]:
def date_columns_to_date_format(df: pd.DataFrame) -> pd.DataFrame:
    for i in date_columns:
        if i in df.columns:
            df[i] = pd.to_datetime(df[i])
    return df

In [7]:
def load_dataset(path: str) -> pd.DataFrame:
    df = pd.read_csv(path)
    df = date_columns_to_date_format(df)
    return df

In [8]:
def make_timeline_for_year(year: int=2023):
    data_1 = np.arange(datetime(year=year, month=1, day=1), datetime(year=year, month=5, day=1), timedelta(hours=1)).tolist()
    data_2 = np.arange(datetime(year=year, month=10, day=1), datetime(year=year+1, month=1, day=1), timedelta(hours=1)).tolist()
    return np.array(data_1+data_2)

In [9]:
def get_weather_page_link(dt: datetime) -> str:
    return f'https://msk.nuipogoda.ru/{dt.day}-{index_month[dt.month]}#{dt.year}'

In [10]:
def get_weather_of_day(i):
    # [0]
    link = get_weather_page_link(i)
    # print(link)
    data = requests.get(link).text
    soup = bs4.BeautifulSoup(data, 'html.parser')
    l = soup.find_all('tr', attrs={'time': True})
    lk = list()
    for i in l[18:26]:
        d = dict()
        d['описание погоды'] = i.find_all('div')[0]['title']
        d['время'] = i.find('span', {'class': 'c1'}).text
        d['температура'] = float(i.find('span', {'class': 'ht'}).text.replace('°', ''))
        d['скорость ветра'] = i.find('span', {'class': 'ws'}).text.replace('м/с', '')
        if d['скорость ветра'] == '':
            d['скорость ветра'] = 0
        else:
            d['скорость ветра'] = float(d['скорость ветра'])
        d['давление'] = float(i.find('span', {'class': 'p'}).text)
        lk.append(d)
    return pd.DataFrame(lk)

# **Loading tables**

In [11]:
turn_on_off_heating = load_dataset(Pathes.path_turn_on_off_heating)
events_for_period_2023 = load_dataset(Pathes.path_events_for_period_2023)
events_for_period_2024 = load_dataset(Pathes.path_events_for_period_2024)
events_for_period_2023_params = load_dataset(Pathes.path_events_for_period_2023_params)
events_for_period_2024_params = load_dataset(Pathes.path_events_for_period_2024_params)


In [12]:
usefull_events = events_for_period_2024_params['Названия строк'].to_list()

In [13]:
events_for_period_2023__only_usefull_events = events_for_period_2023.loc[events_for_period_2023['Наименование'].isin(usefull_events)]
events_for_period_2024__only_usefull_events = events_for_period_2024.loc[events_for_period_2024['Наименование'].isin(usefull_events)]

In [14]:
events_for_period_2023__only_usefull_events

,Наименование,Источник,Дата создания во внешней системе,Дата закрытия,Округ,УНОМ,Адрес,Дата и время завершения события
1,Температура в квартире ниже нормативной,EDC,2023-10-08 12:26:38,2023-10-08 12:37:31.785,ВАО,8171.0,внутригородская территория муниципальный округ...,2023-10-08 17:34:56.829000
2,Температура в квартире ниже нормативной,EDC,2023-10-08 13:22:11,2023-10-08 13:37:44.841,ВАО,8171.0,внутригородская территория муниципальный округ...,2023-10-08 17:35:06.822000
9,Температура в квартире ниже нормативной,EDC,2023-10-09 00:45:37,2023-10-09 00:51:20.340,ВАО,8171.0,внутригородская территория муниципальный округ...,2023-10-09 09:40:25.631000
28,Температура в квартире ниже нормативной,EDC,2023-10-09 12:15:37,2023-10-09 12:38:55.014,ВАО,8171.0,внутригородская территория муниципальный округ...,2023-10-09 13:56:18.985000
37,Температура в квартире ниже нормативной,EDC,2023-10-27 19:49:36,2023-10-27 19:50:43.828,ВАО,8171.0,внутригородская территория муниципальный округ...,2023-10-27 20:20:44.166000
...,...,...,...,...,...,...,...,...
419249,Температура в квартире ниже нормативной,EDC,2023-10-10 20:57:26,2023-10-10 21:05:11.414,ВАО,23362.0,внутригородская территория муниципальный округ...,2023-10-10 21:21:32.527000
419251,Температура в квартире ниже нормативной,EDC,2023-10-11 04:36:22,2023-10-11 04:43:05.636,ВАО,23362.0,внутригородская территория муниципальный округ...,2023-10-11 07:29:21.280000
419254,Температура в квартире ниже нормативной,EDC,2023-10-18 17:12:03,2023-10-18 17:15:04.871,ВАО,24189.0,внутригородская территория муниципальный округ...,2023-10-18 17:15:04.775000
419255,Температура в квартире ниже нормативной,EDC,2023-10-20 18:27:09,2023-10-20 18:30:35.591,ВАО,24189.0,внутригородская территория муниципальный округ...,2023-10-20 18:51:09.680000


# **Get weather**

In [15]:
def get_weather_dataset(year: int):
    weathe_dataset = None
    timeline = make_timeline_for_year(year)
    for i in tqdm(timeline[::24].tolist()):
        df = get_weather_of_day(i)
        # print(get_weather_page_link(i))
        if weathe_dataset is None:
            weathe_dataset = pd.DataFrame(np.repeat(df.values, 3, axis=0), columns=df.columns)
        else:
            dfp = pd.DataFrame(np.repeat(df.values, 3, axis=0), columns=df.columns)
            weathe_dataset = pd.concat([weathe_dataset, dfp])
    weathe_dataset['timeline'] = timeline
    return weathe_dataset


In [ ]:
weathe_dataset_2022 = get_weather_dataset(2022)
weathe_dataset_2023 = get_weather_dataset(2023)

In [19]:
weathe_dataset_2024 = None
timeline = make_timeline_for_year(2024)
for i in tqdm(timeline[::24].tolist()):
    df = get_weather_of_day(i)
    # print(get_weather_page_link(i))
    if weathe_dataset_2024 is None:
        weathe_dataset_2024 = pd.DataFrame(np.repeat(df.values, 3, axis=0), columns=df.columns)
    else:
        dfp = pd.DataFrame(np.repeat(df.values, 3, axis=0), columns=df.columns)
        weathe_dataset_2024 = pd.concat([weathe_dataset_2024, dfp])

100%|██████████| 213/213 [01:49<00:00,  1.95it/s]


In [21]:
weathe_dataset_2024['timeline'] = timeline
weathe_dataset_2024

In [27]:
pd.concat([weathe_dataset_2022, weathe_dataset_2023, weathe_dataset_2024], axis=0).dropna()

,описание погоды,время,температура,скорость ветра,давление,timeline
0,"пасмурно, снег",00:00,-3.0,0.0,738.0,2022-01-01 00:00:00
1,"пасмурно, снег",00:00,-3.0,0.0,738.0,2022-01-01 01:00:00
2,"пасмурно, снег",00:00,-3.0,0.0,738.0,2022-01-01 02:00:00
3,"пасмурно, снег",03:00,-2.0,1.0,737.0,2022-01-01 03:00:00
4,"пасмурно, снег",03:00,-2.0,1.0,737.0,2022-01-01 04:00:00
...,...,...,...,...,...,...
19,"пасмурно, без осадков",18:00,1.0,0.0,750.0,2024-12-31 19:00:00
20,"пасмурно, без осадков",18:00,1.0,0.0,750.0,2024-12-31 20:00:00
21,"пасмурно, без осадков",21:00,1.0,1.0,749.0,2024-12-31 21:00:00
22,"пасмурно, без осадков",21:00,1.0,1.0,749.0,2024-12-31 22:00:00


In [28]:
pd.concat([weathe_dataset_2022, weathe_dataset_2023, weathe_dataset_2024], axis=0).to_csv(Pathes.path_to_weather_dataset, index=False)